**Importación de Librerias**

In [1]:
import tensorflow as tf
from tensorflow import keras
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter

import math
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Descarga y preprocesado de datos**

In [3]:
#Cargar el texto del parlamentario1
text = open("/content/intervencionesAbascal.txt", 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        22573 carácteres
El texto está compuesto de estos 81 carácteres:
['\n', ' ', '!', '%', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '¿', 'Á', 'á', 'é', 'í', 'ñ', 'ó', 'ú', '\u200c', '—', '―', '…']


In [4]:
#Como las redes neuronales solo procesan valores numéricos, no letras. Traduciremos los caracteres a representación numérica.
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '%' :   3,
  ',' :   4,
  '.' :   5,
  '0' :   6,
  '1' :   7,
  '2' :   8,
  '3' :   9,
  '4' :  10,
  '5' :  11,
  '6' :  12,
  '7' :  13,
  '8' :  14,
  '9' :  15,
  ':' :  16,
  ';' :  17,
  '?' :  18,
  'A' :  19,
  'B' :  20,
  'C' :  21,
  'D' :  22,
  'E' :  23,
  'F' :  24,
  'G' :  25,
  'H' :  26,
  'I' :  27,
  'J' :  28,
  'L' :  29,
  'M' :  30,
  'N' :  31,
  'O' :  32,
  'P' :  33,
  'Q' :  34,
  'R' :  35,
  'S' :  36,
  'T' :  37,
  'U' :  38,
  'V' :  39,
  'X' :  40,
  'Y' :  41,
  'Z' :  42,
  'a' :  43,
  'b' :  44,
  'c' :  45,
  'd' :  46,
  'e' :  47,
  'f' :  48,
  'g' :  49,
  'h' :  50,
  'i' :  51,
  'j' :  52,
  'k' :  53,
  'l' :  54,
  'm' :  55,
  'n' :  56,
  'o' :  57,
  'p' :  58,
  'q' :  59,
  'r' :  60,
  's' :  61,
  't' :  62,
  'u' :  63,
  'v' :  64,
  'x' :  65,
  'y' :  66,
  'z' :  67,
  '¡' :  68,
  '¿' :  69,
  'Á' :  70,
  'á' :  71,
  'é' :  72,
  'í' :  73,
  'ñ' :  74,
  'ó' :  75,
  'ú' :  76,

In [5]:
#Ahora pasaremos el texto a un array de enteros
text_as_int = np.array([char2idx[c] for c in text])

In [6]:
#Mostramos los 50 primeros caracteres del texto text_as_init
print ('texto : {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto : 'Señor Sánchez, ¿cómo se atreve usted a hablarme de'
array([36, 47, 74, 57, 60,  1, 36, 71, 56, 45, 50, 47, 67,  4,  1, 69, 45,
       75, 55, 57,  1, 61, 47,  1, 43, 62, 60, 47, 64, 47,  1, 63, 61, 62,
       47, 46,  1, 43,  1, 50, 43, 44, 54, 43, 60, 55, 47,  1, 46, 47])


**Preparación de los datos para entrenar la RNN**

In [7]:
#Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
#A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
#Comprobamos que las sequences son el texto dividido en 101 caracteres (mostramos 10 secuencias)
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas,'
' si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para imped'
'ir que VOX siga cruzando las líneas que dice usted que cruzamos? Conteste también lo que no me ha con'
'testado durante toda esta legislatura: ¿por qué mintió a los españoles prometiéndoles que no pactaría'
' con estos, con esos y con aquellos?  Contésteme a eso y entonces no habrá monólogos, habrá diálogo p'
'olítico, algo que nunca ha habido en esta Cámara por su culpa.\nNada, señor Sánchez, le prestamos los '
'siete segundos que nos quedan para su próxima cumbre bilateral, que seguro que los aprovecha bien. Mu'
'chas gracias.\nSeñor Sánchez, no puede haber barrios ni calles seguras si no hay fronteras seguras. ¿U'
'sted sabe lo que ha pasado en España las últimas semanas? ¿Sabe usted que algunos de los yihadistas d'
'etenidos en Barcelona, que pretendían atentar en nuestro país

In [9]:
#Creamos una función que devolverá el conjunto de datos de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Aplicamos la función a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

In [10]:
#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

Input data:  'Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas'
Target data: 'eñor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas,'
<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [11]:
#Agrupamos los dataset en batches de 64 . Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


**Construcción del modelo RNN**

In [12]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.5))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [13]:
#Creamos el modelo
embedding_dim = 256
rnn_units = 1024

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
#A continuación inspecionamos las dimenciones de los tensores
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 81) # (batch_size, sequence_length, vocab_size)


In [17]:
#Obtenemos una muestra de la distribución de salida
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
print(sampled_indices_characters)

[ 6  0 69 44 17 16  6 26 46  4 28 47 37 50 31 35 42 56 43 78 47 66  3 16
 69 24 36 77 77 47  6 36 35 24 56 66 57 61 69 48 10 53 72 59 15 24 13 33
 43 21 25 60 10 48 73  3 62 33 26 71 74 69 15 46 31 19 20 15 41 77  6  2
 47 10 29 73 75 27 35 45 50 62 72 65 70 27 46 43 56 75  5 62  0 42  4 26
 36 32 18 53]


**Entrenamiento del modelo RNN**

In [19]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [21]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Abascal'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [22]:
def calculate_bleu(reference_text, generated_text):

    """
    Calcula el BLEU score comparando el texto generado con el texto de referencia.
    """

    reference_tokens = [list(reference_text)]
    generated_tokens = list(generated_text)
    smooth_fn = SmoothingFunction().method1
    score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smooth_fn)

    return score



def calculate_perplexity(predictions):
    """
    Calcula la perplejidad promedio de una secuencia basada en los logits de salida del modelo.
    """
    # Convertimos logits a probabilidades con softmax
    probs = tf.nn.softmax(predictions, axis=-1).numpy()

    # Obtenemos las probabilidades máximas (las más probables)
    max_probs = np.max(probs, axis=-1)

    # Calculamos el logaritmo negativo de las probabilidades
    log_probs = -np.log(max_probs + 1e-10)  # Evitamos log(0) con 1e-10

    # Calculamos la perplejidad: exponencial de la media de los logaritmos
    perplexity = np.exp(np.mean(log_probs))

    return perplexity



def compare_word_distribution(reference_text, generated_text):

    """
    Compara la distribución de palabras entre el texto generado y el de referencia.
    """

    ref_counter = Counter(reference_text.split())
    gen_counter = Counter(generated_text.split())
    total_ref = sum(ref_counter.values())
    total_gen = sum(gen_counter.values())

    ref_dist = {word: count / total_ref for word, count in ref_counter.items()}
    gen_dist = {word: count / total_gen for word, count in gen_counter.items()}



    kl_divergence = 0

    for word, prob in gen_dist.items():
        if word in ref_dist:
            kl_divergence += prob * math.log(prob / ref_dist[word])

    return kl_divergence


In [23]:
#Entrenamos el modelo
EPOCHS=200
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 206ms/step - loss: 4.2684
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 4.1852
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step - loss: 3.4605
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - loss: 3.3326
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - loss: 3.2254
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - loss: 3.1866
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - loss: 3.1385
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - loss: 3.0987  
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 202ms/step - loss: 3.0561
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - loss: 3.0188
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - loss: 2.9531
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 2.9001
Epoch 13/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 2.8258
Epoch 14/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 236ms/step - loss: 2.7726
Epoch 15/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - loss: 2.7268
Epoch

In [24]:
model.save("model_Abascal.keras")

In [25]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_Abascal.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

**Generación de texto del modelo RNN**

In [26]:
#Ya que tenemos entrenado los modelos vamos a usarlos para generar texto
#Vamos a reconstruir manualmente los modelos para modifical el batch y su peso para poner el último
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
# Build the model before loading weights
# This creates the necessary layer variables to which weights can be assigned
# Provide an input shape that matches the expected input of your model
# for example:
# input_shape = (1, sequence_length) where sequence_length is the length of your input sequences
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_Abascal.keras")

# Genera logits para una entrada de prueba
sample_input = tf.expand_dims([char2idx[s] for s in "texto de prueba"], axis=0)
logits = model(sample_input)  # Salida sin aplicar softmax

# Calcula la perplejidad con la función corregida
perplexity_value = calculate_perplexity(logits)
print(f"Perplejidad: {perplexity_value}")

Perplejidad: 1.8235864639282227


In [27]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"Hoy en el pleno"))

Hoy en el pleno pero el año 1934, un polper a las familias más necesitadas o para garantizar la soberanía energética. Señor Sánchez, usted combate hasta la filosofía, quizá porque no quiere que los escolares sepan si tiene algún tipo de dato objetivo para sostenerlo o si estamos ante la propaganda vacía que le reprece cuenta de la Corona, señorías, porque ustedes quieren atropellar cualquier tipo de institución democrática, a la que amenazan un día sí y otro también desde la bancada azul del Gobierno: encubierto, un estado de alarma perpetuo con la complicidad y el silencio del Tribunal Constitucional, al que deben tener ustedes, porque lo mataron los sofistas…
Señor Sánchez, ¿respalda el presidente del Gobierno?
Señor Sánchez, ya sabíamos que a usted le gustan los aperitivos, pero que los cobraron sus tolpezas, su sectarismo, y sobre todo que usted no es responsable de ese pasado—: en el año 1934, un golpe de Estado a la República; en el año 1934, un golpe de Estado a la República; en

In [29]:
print(generate_text(model, start_string=u"Es inaceptable que este gobierno haya decidido"))

Es inaceptable que este gobierno haya decididonte esta Cámara. Ustedes tienen domesticado ya a un tribunal y ya sabemos qué quieren hacer con el Cons, por si si tiene algún tipo de dato objetivo para sostenerlo o si estamos ante la propaganda vacía que le reprece cias, que nos ahorren sus delirios, que nos ahorren sus torpezas, su sectarismo, y sobre todo que la fabria nuestro entorno, mucho más grave. Deje de hacerse el imprescindible allí donde nadie le echa de menos hez. Señor Sánchez, usted combate hasta la filosofía, quizá porque no quiere que los escolares sepan con la ruina, señorías, nos traen la división, la ya clásica división por sexos ested al presidente de la República—, sí, porque su Gobierno, señor Sánchez, es una fábrica de miseria, igual que su ideología.
Muchas gracias
Concluyo inmediatamente, presidenta. Vimos ayer cómo el representante de la República—, sí, porque su Gobierno, señor Sánchez, es una fábrica de miseria, igual que su ideología.
Muchas gracias, señor Sá

In [30]:
# Texto de referencia: toma un segmento del dataset original
reference_text = text[:1000]  # Ejemplo: primeros 1000 caracteres del texto original
generated_text = generate_text(model, start_string="Ejemplo de inicio")

# BLEU Score
bleu_score = calculate_bleu(reference_text, generated_text)
print(f"BLEU Score: {bleu_score}")

# Perplejidad
input_eval = [char2idx[s] for s in "Ejemplo de inicio"]
input_eval = tf.expand_dims(input_eval, 0)
predictions = model(input_eval)

perplexity = calculate_perplexity(predictions[0])
print(f"Perplejidad: {perplexity}")

# Comparación de Distribución de Palabras
kl_divergence = compare_word_distribution(reference_text, generated_text)
print(f"KL Divergence (Distribución de Palabras): {kl_divergence}")

BLEU Score: 0.4292349477085166
Perplejidad: 1.5743392705917358
KL Divergence (Distribución de Palabras): 0.0740959554496724
